## INF1608 - Análise Numérica - 2016.1
## Departamento de Informática - PUC-Rio 
## Prof. Hélio Lopes - lopes@inf.puc-rio.br
## http://www.inf.puc-rio.br/~lopes



## Lista 1

1) Modifique a implementação da decomposição LU para incluir a permutação de linhas, obtendo a decomposição PA=LU.

2) Implemente a função LUsolve que resolve o sistema Ax=b, tendo em mão a deomposição PA=LU.

3) Utilize as implementações dos items 1 e 2 da lista para resolver o sistema:

A = np.matrix([[1.,0.,0.,1.],[-1.,1.,0.,1.],[-1.,-1.,1.,1.],[-1.,-1.,-1.,1.]])
e
b = np.array([1.,1.,2.,0.])

4) Faça uma função que verique se uma matriz A de tamanho nxn é estritamente diagonal dominante:
        Definição: Uma matriz A (nxn) é estritamente diagonal dominante se para toda linha i vale:
                   $$|A_{i,i}| > \sum_{i=1, i\ne j}^n |A_{i,j}|$$
                   
5) Implemente o método de Jacobi para a solução de um sistema de equações lineares. Defina um critério de parada para o seu método, e explique-o. 
        https://pt.wikipedia.org/wiki/M%C3%A9todo_de_Jacobi

6) Implemente o método de Gauss-Seidel para a solução de um sistema de equações lineares. Defina um critério de parada para o seu método, e explique-o.
        https://pt.wikipedia.org/wiki/M%C3%A9todo_de_Gauss-Seidel

7) Teste muito bem todos esses algoritmos!

In [122]:
### Questão Número 1 ###
import numpy as np

# Função que retorna o índice da linha com o maior pivô disponível
# de uma matriz de acordo com uma coluna.
#
# A: matriz que terá o pivô procurado
# c: índice da coluna que terá o pivô procurado
def FindMaxPivotLineIndex(A,c):
    
    maxPivot = np.absolute(A[c][c])
    maxPivotLineIndex = c
    
    n = len(A)
    for i in range(c,n):
        
        if np.absolute(A[i][c]) > maxPivot:
            maxPivot = np.absolute(A[i][c])
            maxPivotLineIndex = i        
    
    return maxPivotLineIndex

# Função que retorna uma matriz de permutação.
#
# n: dimenção da matriz quadrada
# l1: índice da primeira linha que será trocada com a segunda
# l2: índice da segunda linha que será trocada com a primeira
def BuildPermutationMatrix(n,l1,l2):
    
    P = np.identity(n)
    
    if l1 >= n or l2 >= n:
        return P
    
    P[l1][l1] = 0.0
    P[l1][l2] = 1.0
    
    P[l2][l2] = 0.0
    P[l2][l1] = 1.0
    
    return P

# Decomposição LU com permutação de linhas.
# A função retorna um vetor com duas posições, que correspondem aos segintes dados:
# índice [0]: Matriz de Permutação;
# índice [1]: Matriz da redução LU;
#
# A: matriz que será decomposta
def LUdecompWithPermutation(A):
    
    LU = np.copy(A)
    n = len(A)
    vetP = np.zeros((n-1,n,n))
    
    for j in range(n-1):
        
        pivotLineIndex = FindMaxPivotLineIndex(LU,j)
        auxP = BuildPermutationMatrix(n,j,pivotLineIndex)
        LU = np.dot(auxP,LU)
        vetP[j] = auxP
        
        for i in range(j+1,n):            
            LU[i][j] = LU[i][j]/LU[j][j]
            for k in range(j+1,n):
                LU[i][k] = LU[i][k] - LU[i][j]*LU[j][k]

    P = reduce(np.dot, vetP[::-1])
    return np.array((P,LU))

### Questão Número 2 ###
def LUforwardsub (L,b):
    n = len(L)
    y = np.zeros(n)
    y[0] = b[0]
    for i in range(1,n):
        y[i] = b[i]
        for j in range(i):
            y[i] = y[i] - L[i][j]*y[j]
    return y
            
def LUbackwardsub (U,y):
    n = len(U)
    x = np.zeros(n)
    x[n-1] = y[n-1]/U[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i] = y[i]
        for j in range(i+1,n):
            x[i] = x[i] - U[i][j]*x[j]
        x[i] /= U[i][i]
    return x

# Solução de um sistema de equações lineares com permutação de linhas.
#
# P: matriz de permutação das linhas
# LU: matriz de decomposição LU
# b: vetor b do sistema (Ax = b)
def LUsolveWithPermutation(P,LU,b):
    b = np.dot(P,b)
    y = LUforwardsub(LU,b)
    x = LUbackwardsub(LU,y)
    return x

A = np.array([[1.0, -3.0, 2.0],[-1.0, 2.0, 6.0],[4.0, -2.0, 1.0]])
b = np.array([2.0,3.0,1.0])
print("Matriz A:")
print(A)
print("\nVetor b:")
print(b)

print("\n-- Redução LU:\n")

PAndLU = LUdecompWithPermutation(A)
print("Matriz de permutação(P):")
print(PAndLU[0])
print("Matriz de decomposição LU:")
print(PAndLU[1])
print("\nSolução:")
print(LUsolveWithPermutation(PAndLU[0],PAndLU[1],b))

Matriz A:
[[ 1. -3.  2.]
 [-1.  2.  6.]
 [ 4. -2.  1.]]

Vetor b:
[ 2.  3.  1.]

-- Redução LU:

Matriz de permutação(P):
[[ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]]
Matriz de decomposição LU:
[[ 4.   -2.    1.  ]
 [ 0.25 -2.5   1.75]
 [-0.25 -0.6   7.3 ]]

Solução:
[-0.04109589 -0.28767123  0.5890411 ]


In [124]:
### Questão Número 3 ###
A = np.matrix([[1.,0.,0.,1.],[-1.,1.,0.,1.],[-1.,-1.,1.,1.],[-1.,-1.,-1.,1.]])
b = np.array([1.,1.,2.,0.])
print("Matriz A:")
print(A)
print("\nVetor b:")
print(b)

print("\n-- Redução LU:\n")

PAndLU = LUdecompWithPermutation(A)
print("Matriz de permutação(P):")
print(PAndLU[0])
print("Matriz de decomposição LU:")
print(PAndLU[1])
print("\nSolução:")
print(LUsolveWithPermutation(PAndLU[0],PAndLU[1],b))

Matriz A:
[[ 1.  0.  0.  1.]
 [-1.  1.  0.  1.]
 [-1. -1.  1.  1.]
 [-1. -1. -1.  1.]]

Vetor b:
[ 1.  1.  2.  0.]

-- Redução LU:

Matriz de permutação(P):
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
Matriz de decomposição LU:
[[ 1.  0.  0.  1.]
 [-1.  1.  0.  2.]
 [-1. -1.  1.  4.]
 [-1. -1. -1.  8.]]

Solução:
[ 0.  0.  1.  1.]


In [126]:
### Questão Número 4 ###
import numpy as np

# Define se uma matriz é estritamente diagonal ou não.
#
# A: matriz que será verificada
def IsEstritamenteDiagonal(A):
    M = np.copy(A)
    for i in range(len(M)):
        acum = 0.0
        for j in range(len(M)):
            if i != j:
                acum = acum + np.absolute(M[i][j])
                
        if acum >= M[i][i]:
            return False
        
    return True

A = np.array(([1.,0.,0.],[1.,4.0,3.],[0.,0.,1.]))
print("Matriz A:")
print(A)
print("\nSolução:")
print(IsEstritamenteDiagonal(A))

print("\n")

A = np.array(([1.,0.,0.],[1.,4.1,3.],[0.,0.,1.]))
print("Matriz A:")
print(A)
print("\nSolução:")
print(IsEstritamenteDiagonal(A))

Matriz A:
[[ 1.  0.  0.]
 [ 1.  4.  3.]
 [ 0.  0.  1.]]

Solução:
False


Matriz A:
[[ 1.   0.   0. ]
 [ 1.   4.1  3. ]
 [ 0.   0.   1. ]]

Solução:
True


In [128]:
### Questão Número 5 ###
import numpy as np
from numpy import linalg as la

# Realiza o método de Jacobi para solucionar
# um sistema de equações lineares.
#
# A: 
def DoJacobi(A,b,N,E):

    # Vetor com chute inicial
    x = np.zeros(len(A))
    x.fill(1.0)
    X = np.copy(x)
    
    D = np.diag(A)
    R = A - np.diagflat(D)
    
    for _ in range(N):
        x = (b - np.dot(R,x))/ D
        if((np.absolute(la.norm(x) - la.norm(X))) < E):
            break
        else:
            X = np.copy(x)
        
    return x

A = np.array([[4.0, -2.0, 1.0], [1.0, -3.0, 2.0], [-1.0, 2.0, 6.0]])
b = np.array([1.0, 2.0, 3.0])
n = 25

x = DoJacobi(A, b, n, 0.00001)
print(x)

[-0.04110016 -0.28766977  0.58904605]


In [127]:
### Questão Número 6 ###
import numpy as np

def DoGaussSeidel(A,b,N,TOL):
    
    # Vetor com chute inicial
    x0 = np.zeros(len(A))
    x0.fill(1.0)
    x = np.copy(x0)
    
    for _ in range(N):
        
        for i in range(len(A)):
            sum = 0
            for j in range(i-1):
                sum = sum + A[i][j]*x[j]
            
            for j in range(i+1,len(A)):
                sum = sum + A[i][j]*x0[j]
                
            x0 = np.copy(x)
            x[i] = (b[i]-sum)/A[i][i]
            
        if((np.absolute(la.norm(x) - la.norm(x0))) < TOL):
            return x
    
    return "Número máximo de iterações excedido."

A = np.array([[4.0, -2.0, 1.0], [1.0, -3.0, 2.0], [-1.0, 2.0, 6.0]])
b = np.array([1.0, 2.0, 3.0])
n = 5

x = DoGaussSeidel(A, b, n, 0.0001)
print(x)
x = DoGaussSeidel(A, b, n, 0.00001)
print(x)

[-0.04214892 -0.33796296  0.49297518]
Número máximo de iterações excedido.
